**Assignment explanation:**

An explanation of the central idea behind your final project (what is the idea?, why is it interesting? which datasets did you need to explore the idea?, how did you download them)
A walk-through of your preliminary data-analysis, addressing
What is the total size of your data? (MB, number of rows, number of variables, etc)
What is the network you will be analyzing? (number of nodes? number of links?, degree distributions, what are node attributes?, etc.)
What is the text you will be analyzing?
How will you tie networks and text together in your paper?


The core idea of our project is to map the ecosystem of Danish music festivals by looking at who plays where, how festivals cluster, and how artist descriptions change across years and genres. This lets us explore how the Danish music scene has evolved over time.

We are especially interested in how festival identities shift. For example, Roskilde originally began as a rock and pop festival, but its lineup today spans many genres. We want to see whether festivals are becoming more similar by booking the same artists, or whether they still serve different audiences and demands. We also want to identify whether there are artists who appear across multiple generations, playing both in our parents’ time and in our own.

To explore this, we are building a dataset based on the largest and oldest festivals in Denmark, including Roskilde Festival, Smukfest, Copenhell, Vig Festival, Distortion, and others. Most of the historical lineups come from festivalhistorik.dk, and any missing years will be collected from festival websites or Wikipedia.

Altogether, this gives us around 250 nodes, where each node is a specific festival in a specific year. Two nodes will be connected if they share artists.

> missing a bit more analysis...


For the text analysis, we will use artist descriptions and the official festival descriptions from 2025. We will compare how closely each festival-year matches the 2025 description, to see how festival identities develop and how their audiences may have shifted across generations.
